<a href="https://colab.research.google.com/github/hardiksiloiya/GANs/blob/main/Variational_autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import keras
from keras.layers import Input,Conv2DTranspose,Conv2D,BatchNormalization,LeakyReLU,Dropout,Flatten,Lambda,Dense,Reshape,Activation
from keras import backend as K
from keras.models import Model
from keras.optimizers import Adam
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf



In [3]:
(x_train,y_train),(x_test,y_test)=mnist.load_data()
x_train=x_train.reshape(60000,28,28,1)
x_test=x_test.reshape(10000,28,28,1)
x_train = x_train.astype(np.float32)
y_train = y_train.astype(np.float32)
x_test=x_test.astype(np.float32)
y_test=y_test.astype(np.float32)

11493376/11490434 [==============================] - 0s 0us/step


In [4]:
def func(inp):
  u,log_var=inp
  epsilon=K.random_normal(shape=K.shape(u), mean=0., stddev=1.)
  return u+K.exp(log_var/2)*epsilon

encoder_input=Input((28,28,1))
x=encoder_input
x=Conv2D(filters=32,kernel_size=3,strides=1,padding='same')(x)
x=BatchNormalization()(x)
x=Dropout(rate=0.25)(x)
x=LeakyReLU()(x)
x=Conv2D(filters=32,kernel_size=3,strides=2,padding='same')(x)
x=BatchNormalization()(x)
x=Dropout(rate=0.25)(x)
x=LeakyReLU()(x)
x=Conv2D(filters=16,kernel_size=3,strides=2,padding='same')(x)
x=BatchNormalization()(x)
x=Dropout(rate=0.25)(x)
x=LeakyReLU()(x)
x=Conv2D(filters=16,kernel_size=3,strides=1,padding='same')(x)
x=BatchNormalization()(x)
x=Dropout(rate=0.25)(x)
x=LeakyReLU()(x)
x=Flatten()(x)
u=Dense(2)(x)
log_var=Dense(2)(x)

encoder_out=Lambda(func)([u,log_var])    #sampling layer
en_model=Model(encoder_input,encoder_out)
#en_model.summary()


In [5]:
decoder_input=Input(2)
y=Dense(1568)(decoder_input)
y=Reshape((7,7,32))(y)
y=Conv2DTranspose(filters=64,kernel_size=3,strides=2,padding='same')(y)
y=BatchNormalization()(y)
y=Dropout(rate=0.25)(y)
y=LeakyReLU()(y)
y=Conv2DTranspose(filters=32,kernel_size=3,strides=2,padding='same')(y)
y=BatchNormalization()(y)
y=Dropout(rate=0.25)(y)
y=LeakyReLU()(y)
y=Conv2DTranspose(filters=32,kernel_size=3,strides=1,padding='same')(y)
y=BatchNormalization()(y)
y=Dropout(rate=0.25)(y)
y=LeakyReLU()(y)
y=Conv2DTranspose(filters=1,kernel_size=3,strides=1,padding='same')(y)
y=BatchNormalization()(y)
y=Dropout(rate=0.25)(y)
y=Activation('sigmoid')(y)
decoder_out=y
de_model=Model(decoder_input,decoder_out)
#de_model.summary()

In [6]:
model=Model(encoder_input,de_model(encoder_out))
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 28, 28, 32)   320         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 28, 28, 32)   128         conv2d[0][0]                     
__________________________________________________________________________________________________
dropout (Dropout)               (None, 28, 28, 32)   0           batch_normalization[0][0]        
____________________________________________________________________________________________

In [23]:
optimizer=Adam(lr=0.0005)
print(u[0][0])
def squareloss(a,b):
  sq_loss= (1000*K.mean(K.square(a-b), axis = [1,2,3]))
  return sq_loss


def kl_div(a,b):
  kl_loss=-0.5*K.sum(1+log_var-K.square(u)-K.exp(log_var),axis=1)
  kl_loss=K.sum(kl_loss,axis=0)
  return kl_loss


def loss_func(a,b):
  return squareloss(a,b)+kl_div(a,b)

model.compile(optimizer=optimizer,loss=squareloss)#,metrics=[u])

KerasTensor(type_spec=TensorSpec(shape=(), dtype=tf.float32, name=None), name='tf.__operators__.getitem_7/strided_slice:0', description="created by layer 'tf.__operators__.getitem_7'")


In [24]:
model.fit(x=x_train,y=x_train,batch_size=32,shuffle=True,epochs=10)

Epoch 1/10
 537/1875 [=======>......................] - ETA: 8s - loss: 7209308.5587

KeyboardInterrupt: ignored

In [ ]:
a=np.random.normal(size=30)
b=np.random.normal(size=30)
c=np.array(list(zip(a,b)))
temp=de_model.predict(c)

plt.scatter(c[:, 0] , c[:, 1], c = 'red', alpha=1, s=20)
plt.show()

fig = plt.figure(figsize=(15, 2))
fig.subplots_adjust(hspace=0.4, wspace=0.4)

for i in range(30):
    ax = fig.add_subplot(2, 15, i+1)
    ax.axis('off')
    ax.text(0.5, -0.35, str(np.round(c[i],1)), fontsize=8, ha='center', transform=ax.transAxes)
    
    ax.imshow(temp[i, :,:,0], cmap = 'Greys')